#### In this tutorial we will implement distillation of complex model's knowledge to simpler models. A complex model called teacher is TabilarAutoMl object. Simpler models called students are BoostCB and BoostLGBM objects.

In [1]:
import numpy as np
from pandas import read_csv
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split

from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.dataset.roles import DatetimeRole
from lightautoml.tasks import Task
from lightautoml.addons.distillation import Distiller

## Some Setups

In [2]:
RANDOM_STATE = 42
TEST_SIZE = 2000
TARGET_NAME = 'TARGET'

np.random.seed(RANDOM_STATE)

## Data loading and preparation

In [3]:
data = read_csv('example_data/test_data_files/sampled_app_train.csv')

data['BIRTH_DATE'] = (np.datetime64('2018-01-01') + data['DAYS_BIRTH'].astype(np.dtype('timedelta64[D]'))).astype(str)
data['EMP_DATE'] = (np.datetime64('2018-01-01') + np.clip(data['DAYS_EMPLOYED'], None, 0).astype(np.dtype('timedelta64[D]'))
                    ).astype(str)

data['report_dt'] = np.datetime64('2018-01-01')
data['constant'] = 1
data['allnan'] = np.nan
data.drop(['DAYS_BIRTH', 'DAYS_EMPLOYED'], axis=1, inplace=True)

train, test = train_test_split(data, test_size=2000, random_state=RANDOM_STATE)

In [4]:
train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,BIRTH_DATE,EMP_DATE,report_dt,constant,allnan
9254,445173,1,Cash loans,F,N,Y,0,162000.0,619254.0,29920.5,...,0.0,0.0,0.0,0.0,4.0,1986-04-30,2015-04-26,2018-01-01,1,NaN
1561,120419,0,Cash loans,F,N,Y,1,202500.0,805536.0,36531.0,...,0.0,0.0,0.0,0.0,2.0,1967-06-09,2006-07-18,2018-01-01,1,NaN
1670,266087,0,Cash loans,F,Y,Y,0,157500.0,505332.0,35298.0,...,0.0,0.0,0.0,0.0,1.0,1966-10-05,2012-04-19,2018-01-01,1,NaN
6087,200021,0,Cash loans,F,N,Y,2,202500.0,260640.0,27499.5,...,0.0,1.0,0.0,0.0,2.0,1988-08-22,2009-08-02,2018-01-01,1,NaN
6669,120701,0,Cash loans,F,N,Y,0,54000.0,269550.0,13891.5,...,NaN,NaN,NaN,NaN,NaN,1966-12-08,2015-05-05,2018-01-01,1,NaN


## Binary

### AutoML and distiller creation

In [5]:
roles = {'target': 'TARGET',
         DatetimeRole(base_date=True, seasonality=(), base_feats=False): 'report_dt'}

task = Task('binary')

automl = TabularAutoML(task=task, timeout=30, general_params={'verbose': 0})
distiller = Distiller(automl)

### Distiller fitting and evaluation

In [6]:
distiller.fit(train, roles=roles)
test_pred = distiller.predict(test)
print('Teacher TEST ROC AUC: {}'.format(roc_auc_score(test[roles['target']].values, test_pred.data[:, 0])))

Start automl preset with listed constraints:
- time: 30 seconds
- cpus: 4 cores
- memory: 16 gb

Train data shape: (8000, 125)
Feats was rejected during automatic roles guess: []


Layer 1 ...
Train process start. Time left 27.26897144317627 secs
Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====

Linear model: C = 1e-05 score = 0.6881245485582443
Linear model: C = 5e-05 score = 0.7024667556992085
Linear model: C = 0.0001 score = 0.7151668840515695
Linear model: C = 0.0005 score = 0.7337182172012949
Linear model: C = 0.001 score = 0.739543439739346
Linear model: C = 0.005 score = 0.7409360888385411
Linear model: C = 0.01 score = 0.7397201191026768
Linear model: C = 0.05 score = 0.732730891347388

===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====

Linear model: C = 1e-05 score = 0.6807299975576676
Linear model: C = 5e-05 score = 0.696210227656556
Linear model: C = 0.0001 score = 0.7106043993161469
Line

Time limit exceeded after calculating fold 3


Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
Time left 24.55259609222412
Start fitting Selector_LightGBM ...

===== Start working with fold 0 for Selector_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's auc: 0.716936
[200]	valid's auc: 0.723154
[300]	valid's auc: 0.725056
[400]	valid's auc: 0.724572
Early stopping, best iteration is:
[273]	valid's auc: 0.725929
Selector_LightGBM fitting and predicting completed
Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_0_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's auc: 0.720878
[200]	valid's auc: 0.727493
[300]	valid's auc: 0.726942
[400]	valid's auc: 0.726672
Early stopping, best iteration is:
[253]	valid's auc: 0.730949

===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_0_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's auc: 0.768529
[200]	va

Time limit exceeded after calculating fold 1


[600]	valid's auc: 0.765016
Early stopping, best iteration is:
[421]	valid's auc: 0.770073
Lvl_0_Pipe_1_Mod_0_LightGBM fitting and predicting completed
Optuna may run 1 secs


Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer


Start fitting Lvl_0_Pipe_1_Mod_1_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_1_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's auc: 0.723388
[200]	valid's auc: 0.725825
[300]	valid's auc: 0.722421
Early stopping, best iteration is:
[198]	valid's auc: 0.72662
Lvl_0_Pipe_1_Mod_1_LightGBM fitting and predicting completed
Start fitting Lvl_0_Pipe_1_Mod_2_CatBoost ...

===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_2_CatBoost =====

0:	learn: 0.5835376	test: 0.5714928	best: 0.5714928 (0)	total: 4.03ms	remaining: 20.1s
100:	learn: 0.8525499	test: 0.7422716	best: 0.7436227 (98)	total: 329ms	remaining: 15.9s
200:	learn: 0.9063686	test: 0.7280801	best: 0.7447919 (132)	total: 624ms	remaining: 14.9s
bestTest = 0.7447918653
bestIteration = 132
Shrink model to first 133 iterations.

===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_2_CatBoost =====

0:	learn: 0.5901767	test: 0.5479762	best: 0.5479762 (0)	total: 3.26ms	remai

Time limit exceeded in one of the tasks. AutoML will blend level 1 models.


Blending: Optimization starts with equal weights and score 0.74467079797947
Blending, iter 0: score = 0.7456198467867428, weights = [0.41915822 0.13035522 0.18183367 0.26865286]
Blending, iter 1: score = 0.7456691777588502, weights = [0.39007956 0.13289087 0.20425488 0.2727747 ]
Blending, iter 2: score = 0.7456828114241374, weights = [0.38312432 0.1467195  0.20061296 0.26954323]
Blending, iter 3: score = 0.7456828114241374, weights = [0.38312432 0.1467195  0.20061296 0.26954323]
No score update. Terminated

Automl preset training completed in 24.85 seconds.
Teacher TEST ROC AUC: 0.7455376893763362


### Evaluation of the students on true labels

In [7]:
distiller.distill(train, labels=train['TARGET'])

metrics = distiller.eval_metrics(test, metrics=[roc_auc_score, accuracy_score])
metrics

Train data shape: (8000, 125)
Feats was rejected during automatic roles guess: []


Layer 1 ...
Train process start. Time left 9999999998.769627 secs
Start fitting Lvl_0_Pipe_0_Mod_0_CatBoost ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_CatBoost =====

0:	test: 0.5324908	best: 0.5324908 (0)	total: 2.24ms	remaining: 6.73s
100:	test: 0.7027526	best: 0.7089207 (80)	total: 176ms	remaining: 5.05s
200:	test: 0.7010637	best: 0.7110305 (120)	total: 351ms	remaining: 4.89s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7110305084
bestIteration = 120

Shrink model to first 121 iterations.

===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_CatBoost =====

0:	test: 0.5413430	best: 0.5413430 (0)	total: 1.8ms	remaining: 5.4s
100:	test: 0.7273110	best: 0.7277007 (98)	total: 174ms	remaining: 5s
200:	test: 0.7406763	best: 0.7421053 (195)	total: 348ms	remaining: 4.85s
300:	test: 0.7450725	best: 0.7452751 (298)	total: 522ms	remaining: 4.68s
Stopped by overfitting 

,roc_auc_score,accuracy_score
Lvl_0_Pipe_0_Mod_0_CatBoost,0.730231,0.9275
Lvl_0_Pipe_0_Mod_0_LightGBM,0.730428,0.9275


### Teacher knowledge distillation

In [8]:
automl = TabularAutoML(task=task, timeout=30, verbose=0)
distiller = Distiller(automl)
distiller.fit(train, roles=roles)
best_model = distiller.distill(train)
print('Best model after distillation: {}'.format(best_model.levels[0][0].ml_algos[0].name))

Best model after distillation: Lvl_0_Pipe_0_Mod_0_LightGBM


### Evaluation of the students on labels derived from teacher

In [9]:
metrics = distiller.eval_metrics(test)
metrics

,roc_auc_score
Lvl_0_Pipe_0_Mod_0_CatBoost,0.582225
Lvl_0_Pipe_0_Mod_0_LightGBM,0.576881


## Multiclass

### Create fake multiclass target

In [10]:
data[TARGET_NAME] = np.where(np.random.rand(data.shape[0]) > .5, 2, data[TARGET_NAME].values)
data[TARGET_NAME].value_counts()

2    4924
0    4677
1     399
Name: TARGET, dtype: int64

In [11]:
train_dist, test_dist = train_test_split(data, test_size=2000, random_state=RANDOM_STATE)

In [12]:
train_dist.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,BIRTH_DATE,EMP_DATE,report_dt,constant,allnan
9254,445173,2,Cash loans,F,N,Y,0,162000.0,619254.0,29920.5,...,0.0,0.0,0.0,0.0,4.0,1986-04-30,2015-04-26,2018-01-01,1,NaN
1561,120419,0,Cash loans,F,N,Y,1,202500.0,805536.0,36531.0,...,0.0,0.0,0.0,0.0,2.0,1967-06-09,2006-07-18,2018-01-01,1,NaN
1670,266087,0,Cash loans,F,Y,Y,0,157500.0,505332.0,35298.0,...,0.0,0.0,0.0,0.0,1.0,1966-10-05,2012-04-19,2018-01-01,1,NaN
6087,200021,0,Cash loans,F,N,Y,2,202500.0,260640.0,27499.5,...,0.0,1.0,0.0,0.0,2.0,1988-08-22,2009-08-02,2018-01-01,1,NaN
6669,120701,0,Cash loans,F,N,Y,0,54000.0,269550.0,13891.5,...,NaN,NaN,NaN,NaN,NaN,1966-12-08,2015-05-05,2018-01-01,1,NaN


### AutoML and distiller creation

In [13]:
roles = {'target': 'TARGET',
         DatetimeRole(base_date=True, seasonality=(), base_feats=False): 'report_dt'}

task = Task('multiclass')

automl = TabularAutoML(task=task, timeout=30, general_params={'verbose': 0})
distiller = Distiller(automl)

### Distiller fitting and evaluation

In [14]:
distiller.fit(train_dist, roles=roles)
test_pred = distiller.predict(test_dist)

Start automl preset with listed constraints:
- time: 30 seconds
- cpus: 4 cores
- memory: 16 gb

Train data shape: (8000, 125)
Feats was rejected during automatic roles guess: []


Layer 1 ...
Train process start. Time left 29.19017004966736 secs
Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====

Linear model: C = 1e-05 score = -0.8310585533827544
Linear model: C = 5e-05 score = -0.8288064428791404
Linear model: C = 0.0001 score = -0.8270068503916264
Linear model: C = 0.0005 score = -0.8225184968672693
Linear model: C = 0.001 score = -0.8221663088351489
Linear model: C = 0.005 score = -0.826257590688765
Linear model: C = 0.01 score = -0.8307471354492009


Time limit exceeded after calculating fold 0


Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
Time left 26.87898015975952
Start fitting Selector_LightGBM ...

===== Start working with fold 0 for Selector_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's multi_logloss: 0.823935
[200]	valid's multi_logloss: 0.823738
[300]	valid's multi_logloss: 0.82497
Early stopping, best iteration is:
[171]	valid's multi_logloss: 0.823048
Selector_LightGBM fitting and predicting completed
Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_0_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's multi_logloss: 0.821382
[200]	valid's multi_logloss: 0.818443
[300]	valid's multi_logloss: 0.817912
[400]	valid's multi_logloss: 0.817308
[500]	valid's multi_logloss: 0.817757


Time limit exceeded after calculating fold 0


[600]	valid's multi_logloss: 0.819518
Early stopping, best iteration is:
[404]	valid's multi_logloss: 0.817228
Lvl_0_Pipe_1_Mod_0_LightGBM fitting and predicting completed
Start fitting Lvl_0_Pipe_1_Mod_2_CatBoost ...

===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_2_CatBoost =====

0:	learn: 1.0867660	test: 1.0870159	best: 1.0870159 (0)	total: 9.7ms	remaining: 29.1s
100:	learn: 0.8005864	test: 0.8284995	best: 0.8284995 (100)	total: 278ms	remaining: 7.97s
200:	learn: 0.7637110	test: 0.8237872	best: 0.8237872 (200)	total: 505ms	remaining: 7.04s
300:	learn: 0.7319800	test: 0.8213015	best: 0.8213015 (300)	total: 755ms	remaining: 6.77s
400:	learn: 0.7011366	test: 0.8227771	best: 0.8209937 (316)	total: 994ms	remaining: 6.44s
bestTest = 0.8209936523
bestIteration = 316
Shrink model to first 317 iterations.

===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_2_CatBoost =====

0:	learn: 1.0868762	test: 1.0869861	best: 1.0869861 (0)	total: 2.78ms	remaining: 8.34s
100:	learn: 0.7992822	t

Time limit exceeded after calculating fold 2


Lvl_0_Pipe_1_Mod_2_CatBoost fitting and predicting completed
Optuna may run 1 secs
Start fitting Lvl_0_Pipe_1_Mod_3_CatBoost ...

===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_3_CatBoost =====

0:	learn: 1.0868690	test: 1.0870472	best: 1.0870472 (0)	total: 2.4ms	remaining: 7.18s
100:	learn: 0.8105096	test: 0.8291441	best: 0.8291441 (100)	total: 188ms	remaining: 5.39s
200:	learn: 0.7845699	test: 0.8231129	best: 0.8229466 (198)	total: 364ms	remaining: 5.08s
300:	learn: 0.7628251	test: 0.8209306	best: 0.8209306 (300)	total: 542ms	remaining: 4.86s
400:	learn: 0.7427934	test: 0.8190285	best: 0.8189991 (370)	total: 713ms	remaining: 4.62s
500:	learn: 0.7239160	test: 0.8186577	best: 0.8181474 (449)	total: 882ms	remaining: 4.4s
bestTest = 0.8181474304
bestIteration = 449
Shrink model to first 450 iterations.
Lvl_0_Pipe_1_Mod_3_CatBoost fitting and predicting completed
Start fitting Lvl_0_Pipe_1_Mod_3_CatBoost ...

===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_3_CatBoost =====

0:	

Time limit exceeded in one of the tasks. AutoML will blend level 1 models.


Blending: Optimization starts with equal weights and score -0.8198032969944179
Blending, iter 0: score = -0.819250144213438, weights = [0.24204701 0.592286   0.         0.16566704]
Blending, iter 1: score = -0.8192373153828084, weights = [0.3058406  0.52807516 0.         0.16608426]
Blending, iter 2: score = -0.8192373143211007, weights = [0.30492693 0.5286782  0.         0.16639492]
Blending, iter 3: score = -0.8192373127974569, weights = [0.3048527  0.5287929  0.         0.16635442]
Blending, iter 4: score = -0.8192373108118772, weights = [0.30483687 0.5297537  0.         0.16540949]

Automl preset training completed in 21.99 seconds.


In [15]:
print('Teacher TEST LOG_LOSS: {}'.format(log_loss(test_dist[roles['target']].values, test_pred.data)))

Teacher TEST LOG_LOSS: 0.8271531527936459


### Evaluation of the students on true labels

In [16]:
distiller.distill(train_dist, labels=train_dist['TARGET'])

metrics = distiller.eval_metrics(test_dist)
metrics

Train data shape: (8000, 125)
Feats was rejected during automatic roles guess: []


Layer 1 ...
Train process start. Time left 9999999998.197756 secs
Start fitting Lvl_0_Pipe_0_Mod_0_CatBoost ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_CatBoost =====

0:	learn: 1.0869596	test: 1.0870764	best: 1.0870764 (0)	total: 2.65ms	remaining: 7.96s
100:	learn: 0.8123933	test: 0.8304814	best: 0.8304814 (100)	total: 245ms	remaining: 7.04s
200:	learn: 0.7849359	test: 0.8256576	best: 0.8256576 (200)	total: 482ms	remaining: 6.72s
300:	learn: 0.7610346	test: 0.8243072	best: 0.8243019 (299)	total: 717ms	remaining: 6.43s
400:	learn: 0.7333857	test: 0.8230621	best: 0.8228896 (390)	total: 961ms	remaining: 6.22s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8228896423
bestIteration = 390

Shrink model to first 391 iterations.

===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_CatBoost =====

0:	learn: 1.0869318	test: 1.0871622	best: 1.0871622 (0)	total: 2.58ms	rema

,log_loss
Lvl_0_Pipe_0_Mod_0_CatBoost,0.827001
Lvl_0_Pipe_0_Mod_0_LightGBM,0.828566


### Teacher knowledge distillation

In [17]:
automl = TabularAutoML(task=task, timeout=30, verbose=0)
distiller = Distiller(automl)
distiller.fit(train_dist, roles=roles)
best_model = distiller.distill(train_dist)
print('Best model after distillation: {}'.format(best_model.levels[0][0].ml_algos[0].name))

Best model after distillation: Lvl_0_Pipe_0_Mod_0_CatBoost


### Evaluation of the students on labels derived from teacher

In [18]:
metrics = distiller.eval_metrics(test_dist)
metrics

,log_loss
Lvl_0_Pipe_0_Mod_0_CatBoost,1.486785
Lvl_0_Pipe_0_Mod_0_LightGBM,1.257099


## Regression

### Split data

In [19]:
train_reg, test_reg = train_test_split(data, test_size=2000, random_state=RANDOM_STATE)

### AutoML and distiller creation

In [20]:
roles = {'target': 'TARGET',
         DatetimeRole(base_date=True, seasonality=(), base_feats=False): 'report_dt'}

task = Task('reg')

automl = TabularAutoML(task=task, timeout=30, general_params={'verbose': 0})
distiller = Distiller(automl)

### Distiller fitting and evaluation

In [21]:
distiller.fit(train_reg, roles=roles)
test_pred = distiller.predict(test_reg)

Start automl preset with listed constraints:
- time: 30 seconds
- cpus: 4 cores
- memory: 16 gb

Train data shape: (8000, 125)
Feats was rejected during automatic roles guess: []


Layer 1 ...
Train process start. Time left 29.419092416763306 secs
Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====

Linear model: C = 1e-05 score = -0.9547608204588335
Linear model: C = 5e-05 score = -0.955419268191003
Linear model: C = 0.0001 score = -0.9569700017582619

===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====

Linear model: C = 1e-05 score = -0.9593059073405834
Linear model: C = 5e-05 score = -0.9593806903493365
Linear model: C = 0.0001 score = -0.9603844330374488

===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====

Linear model: C = 1e-05 score = -0.9545675256697337
Linear model: C = 5e-05 score = -0.9546683029346604
Linear model: C = 0.0001 score = -0.9560513404303745

===== Start working

0:	learn: 0.9799124	test: 0.9783208	best: 0.9783208 (0)	total: 1.95ms	remaining: 3.9s
100:	learn: 0.9506027	test: 0.9822018	best: 0.9782478 (2)	total: 125ms	remaining: 2.35s
200:	learn: 0.9243658	test: 0.9849048	best: 0.9782478 (2)	total: 250ms	remaining: 2.24s
300:	learn: 0.8967586	test: 0.9880751	best: 0.9782478 (2)	total: 377ms	remaining: 2.13s
bestTest = 0.9782477646
bestIteration = 2
Shrink model to first 3 iterations.
Lvl_0_Pipe_1_Mod_2_CatBoost fitting and predicting completed
Optuna may run 6.423104286193848 secs
Start fitting Lvl_0_Pipe_1_Mod_3_CatBoost ...

===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_3_CatBoost =====

0:	learn: 0.9800498	test: 0.9780373	best: 0.9780373 (0)	total: 1.62ms	remaining: 3.24s
100:	learn: 0.9603680	test: 0.9821764	best: 0.9779746 (1)	total: 109ms	remaining: 2.05s
200:	learn: 0.9448291	test: 0.9837625	best: 0.9779746 (1)	total: 217ms	remaining: 1.94s
300:	learn: 0.9285148	test: 0.9856668	best: 0.9779746 (1)	total: 323ms	remaining: 1.82s
best

0:	learn: 0.9793591	test: 0.9811960	best: 0.9811960 (0)	total: 1.77ms	remaining: 5.32s
100:	learn: 0.9635357	test: 0.9803982	best: 0.9801357 (78)	total: 132ms	remaining: 3.78s
200:	learn: 0.9498822	test: 0.9811126	best: 0.9800638 (139)	total: 263ms	remaining: 3.67s
bestTest = 0.9800638015
bestIteration = 139
Shrink model to first 140 iterations.

===== Start working with fold 2 for Lvl_0_Pipe_1_Mod_3_CatBoost =====

0:	learn: 0.9798781	test: 0.9789418	best: 0.9789418 (0)	total: 1.85ms	remaining: 5.55s
100:	learn: 0.9634209	test: 0.9774013	best: 0.9773156 (84)	total: 133ms	remaining: 3.82s
200:	learn: 0.9498537	test: 0.9779275	best: 0.9769329 (136)	total: 261ms	remaining: 3.63s
bestTest = 0.9769328985
bestIteration = 136
Shrink model to first 137 iterations.

===== Start working with fold 3 for Lvl_0_Pipe_1_Mod_3_CatBoost =====

0:	learn: 0.9787547	test: 0.9839140	best: 0.9839140 (0)	total: 1.68ms	remaining: 5.03s
100:	learn: 0.9623116	test: 0.9824025	best: 0.9824025 (100)	total: 123ms	

In [22]:
print('Teacher TEST MSE: {}'.format(mean_squared_error(test_reg[roles['target']].values, test_pred.data)))

Teacher TEST MSE: 0.9562062136554276


### Evaluation of the students on true labels

In [23]:
distiller.distill(train_reg, labels=train_reg['TARGET'])

metrics = distiller.eval_metrics(test_reg, metrics=[mean_squared_error, accuracy_score])
metrics

Train data shape: (8000, 125)
Feats was rejected during automatic roles guess: []


Layer 1 ...
Train process start. Time left 9999999998.825073 secs
Start fitting Lvl_0_Pipe_0_Mod_0_CatBoost ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_CatBoost =====

0:	learn: 0.9801319	test: 0.9780208	best: 0.9780208 (0)	total: 962us	remaining: 2.88s
100:	learn: 0.9649165	test: 0.9790345	best: 0.9779841 (36)	total: 93ms	remaining: 2.67s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9779841448
bestIteration = 36

Shrink model to first 37 iterations.

===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_CatBoost =====

0:	learn: 0.9793639	test: 0.9811610	best: 0.9811610 (0)	total: 970us	remaining: 2.91s
100:	learn: 0.9637700	test: 0.9804590	best: 0.9803626 (89)	total: 93.1ms	remaining: 2.67s
200:	learn: 0.9502095	test: 0.9806501	best: 0.9802950 (126)	total: 185ms	remaining: 2.58s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9802949583
bes

,mean_squared_error,accuracy_score
Lvl_0_Pipe_0_Mod_0_CatBoost,0.956264,0.042
Lvl_0_Pipe_0_Mod_0_LightGBM,0.956839,0.042


### Teacher knowledge distillation

In [24]:
automl = TabularAutoML(task=task, timeout=30, verbose=0)
distiller = Distiller(automl)
distiller.fit(train_reg, roles=roles)
best_model = distiller.distill(train_reg)
print('Best model after distillation: {}'.format(best_model.levels[0][0].ml_algos[0].name))

Best model after distillation: Lvl_0_Pipe_0_Mod_0_LightGBM


### Evaluation of the students on labels derived from teacher

In [25]:
metrics = distiller.eval_metrics(test_reg, metrics=[mean_squared_error, accuracy_score])
metrics

,mean_squared_error,accuracy_score
Lvl_0_Pipe_0_Mod_0_CatBoost,0.956773,0.042
Lvl_0_Pipe_0_Mod_0_LightGBM,0.95706,0.042
